### The benchmark of SARCOS dataset

The performance of different algorithms on this dataset can be found in: https://github.com/Kaixhin/SARCOS

#### Ridge regression with polynomial(3) + fourier([0.1:10:20]) feature: 1.1988,  
NN feature not working.

In [1]:
from aslearn.data.get_data import robot_inv_data
import numpy as np
import torch as th
device = "cuda" if th.cuda.is_available() else "cpu"

Loss = th.nn.MSELoss()
X_train, Y_train, X_val, Y_val, X_test, Y_test = robot_inv_data()

In [2]:
X_train.shape

(40000, 21)

In [3]:
X, Y = np.concatenate([X_train, X_val], axis=0), np.concatenate([Y_train, Y_val], axis=0)
from aslearn.feature.global_features import PolynomialFT
from aslearn.feature.bellcurve import BellCurve
from aslearn.feature.global_features import FourierFT
from aslearn.parametric.ridge import RidgeReg

f1 = PolynomialFT(degree=3) # sklearn.preprocessing.PolynomialFeature is much faster when len(X) is very large !
# f2 = FourierFT(degree=[1.])
X = X.astype(np.float64)
X_test = X_test.astype(np.float64)
X_f = f1(X)#np.concatenate([f1(X),f2(X)], axis=1)
X_test_f = f1(X_test)#np.concatenate([f1(X_test),f2(X_test)], axis=1)
print("feature dim: ", X_f.shape[1])

feature dim:  2024


In [4]:
X_f_t, Y_t, X_test_f_t, Y_t_test = \
th.from_numpy(X_f).to(device).double(), th.from_numpy(Y).to(device).double(),\
th.from_numpy(X_test_f).to(device).double(), th.from_numpy(Y_test).to(device).double()
rr = RidgeReg().fit(X_f_t, Y_t)
pred = rr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

  0%|                                                    | 0/10 [00:00<?, ?it/s]


KeyboardInterrupt: 

VRVM: 1.3459, maximum likelihood linear reg: 1.2800

In [11]:
%%time
from aslearn.parametric.ml_lr import ML_LR
blr = ML_LR(nx=2024, ny=7).fit(X_f_t, Y_t, max_iter=10)
pred = blr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

Inter:  1  Curr_tol:  2765.6472970507034
Inter:  2  Curr_tol:  0.009842628613114357
Inter:  3  Curr_tol:  0.005301166791468859
Inter:  4  Curr_tol:  0.005824516527354717
Inter:  5  Curr_tol:  0.004652787931263447
Inter:  6  Curr_tol:  0.013454971369355917
Inter:  7  Curr_tol:  0.00451575790066272
Inter:  8  Curr_tol:  0.005873679183423519
Inter:  9  Curr_tol:  0.005632089916616678
Inter:  10  Curr_tol:  0.006531859282404184
tensor(1.2800, device='cuda:0', dtype=torch.float64)
CPU times: user 5.27 s, sys: 0 ns, total: 5.27 s
Wall time: 5.26 s


In [2]:
X_train, Y_train, X_val, Y_val, X_test, Y_test =\
    th.from_numpy(X_train).to(device).double(), th.from_numpy(Y_train).to(device).double(), th.from_numpy(X_val).to(device).double(), \
    th.from_numpy(Y_val).to(device).double(), th.from_numpy(X_test).to(device).double(), th.from_numpy(Y_test).to(device).double()
    
from torch.optim import Adam
from torch.nn import MSELoss
from aslearn.parametric.mlp import MLP

#### Neural network: 1.469

In [6]:
param = {"layer":4, "nodes":[21, 500, 500, 7], "actfunc":["ReLU", "ReLU", None], "batch":128, "lr":1e-3, "decay":0.}
net = MLP(param).to(device).double()
print(net)
Loss = MSELoss()
parameters = MLP.setParams(net, param["decay"])
optimizer = Adam(parameters, lr=param["lr"])
for _ in range(100):
    for i in range(len(X_train)//param["batch"]):
        optimizer.zero_grad()
        index = th.randperm(len(X_train))
        curr_index = index[i*param["batch"]:(i+1)*param["batch"]]
        X_b = X_train[curr_index]
        Y_b = Y_train[curr_index]
        pred = net(X_b)
        L = Loss(pred, Y_b)
        L.backward()
        optimizer.step()
    with th.no_grad():
        pred_val = net(X_val)
        L_val = Loss(pred_val, Y_val)
        print("Curr validation loss: ", L_val)
net.eval()
pred_test = net(X_test)
print(Loss(pred_test, Y_test))

MLP(
  (net): Sequential(
    (0): Linear(in_features=21, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=7, bias=True)
  )
)
Curr validation loss:  tensor(23.1734, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(18.8395, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(14.8275, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(13.2116, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(12.2317, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(10.8721, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.9622, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(10.9913, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.5698, device='cuda:0', dtype=torch.float64)
Curr validation loss:  tensor(9.7632, device='cuda:0', dtype=t

#### subsample GPR with hyperparameters optimization: 2.887

In [3]:
from aslearn.nonparametric.gpr import ExactGPR
from aslearn.kernel.kernels import White, Matern

In [5]:
X_test, Y_test = X_test.double(), Y_test.double()
perm = th.randperm(X_train.size(0))
idx = perm[:3000]
X_train_, Y_train_ = X_train[idx].double(), Y_train[idx].double()

In [7]:
l = np.ones((21,7)) * 100.
sigma = np.ones((7,)) * 1.
c = np.ones((7,)) * 1e-6

kernel = White(c=c, dim_in=21, dim_out=7) + Matern(l=l, sigma=sigma, mu=2.5, dim_in=21, dim_out=7)
gpr = ExactGPR(kernel=kernel)
gpr.fit(X_train_, Y_train_, call_hyper_opt=True, solver="Adam", adam_tolerance=1e-5, adam_batch_size=64)

Step:  100 Current evidence: -12027.72
Step:  200 Current evidence: -15308.35
Step:  300 Current evidence: -18122.23
Step:  400 Current evidence: -17258.64
Step:  500 Current evidence: -13866.47
Step:  600 Current evidence: -13648.89
Step:  700 Current evidence: -13921.32
Step:  800 Current evidence: -16706.18
Step:  900 Current evidence: -12643.77
Step:  1000 Current evidence: -12013.69
Step:  1100 Current evidence: -15788.48
Step:  1200 Current evidence: -12152.02
Step:  1300 Current evidence: -13904.80
Step:  1400 Current evidence: -11431.40
Step:  1500 Current evidence: -11305.77
Step:  1600 Current evidence: -17427.09
Step:  1700 Current evidence: -13249.40
Step:  1800 Current evidence: -14701.76
Step:  1900 Current evidence: -11861.40
Step:  2000 Current evidence: -13667.09
Step:  2100 Current evidence: -13001.84
Step:  2200 Current evidence: -11222.64
Step:  2300 Current evidence: -9233.24
Step:  2400 Current evidence: -11248.33
Step:  2500 Current evidence: -10862.03
Step:  260

Step:  21000 Current evidence: -242.22
Step:  21100 Current evidence: -226.98
Step:  21200 Current evidence: -320.51
Step:  21300 Current evidence: -236.90
Step:  21400 Current evidence: -252.03
Step:  21500 Current evidence: -235.42
Step:  21600 Current evidence: -277.05
Step:  21700 Current evidence: -236.82
Step:  21800 Current evidence: -221.21
Step:  21900 Current evidence: -247.21
Step:  22000 Current evidence: -271.40
Step:  22100 Current evidence: -273.12
Step:  22200 Current evidence: -225.60
Step:  22300 Current evidence: -213.60
Step:  22400 Current evidence: -222.83
Step:  22500 Current evidence: -247.51
Step:  22600 Current evidence: -211.82
Step:  22700 Current evidence: -213.51
Step:  22800 Current evidence: -217.26
Step:  22900 Current evidence: -214.83
Step:  23000 Current evidence: -212.92
Step:  23100 Current evidence: -211.42
Step:  23200 Current evidence: -215.14
Step:  23300 Current evidence: -205.82
Step:  23400 Current evidence: -230.45
Step:  23500 Current evid

Step:  42100 Current evidence: -135.47
Step:  42200 Current evidence: -146.55
Step:  42300 Current evidence: -137.23
Step:  42400 Current evidence: -130.73
Step:  42500 Current evidence: -138.05
Step:  42600 Current evidence: -147.82
Step:  42700 Current evidence: -139.56
Step:  42800 Current evidence: -140.85
Step:  42900 Current evidence: -139.43
Step:  43000 Current evidence: -144.44
Step:  43100 Current evidence: -131.62
Step:  43200 Current evidence: -135.04
Step:  43300 Current evidence: -136.42
Step:  43400 Current evidence: -131.66
Step:  43500 Current evidence: -141.53
Step:  43600 Current evidence: -141.98
Step:  43700 Current evidence: -135.77
Step:  43800 Current evidence: -137.13
Step:  43900 Current evidence: -140.23
Step:  44000 Current evidence: -145.08
Step:  44100 Current evidence: -138.39
Step:  44200 Current evidence: -143.46
Step:  44300 Current evidence: -148.35
Step:  44400 Current evidence: -129.62
Step:  44500 Current evidence: -132.78
Step:  44600 Current evid

Step:  63200 Current evidence: -132.48
Step:  63300 Current evidence: -137.75
Step:  63400 Current evidence: -138.77
Step:  63500 Current evidence: -135.49
Step:  63600 Current evidence: -130.04
Step:  63700 Current evidence: -136.85
Step:  63800 Current evidence: -139.62
Step:  63900 Current evidence: -132.93
Step:  64000 Current evidence: -131.86
Step:  64100 Current evidence: -132.74
Step:  64200 Current evidence: -141.30
Step:  64300 Current evidence: -137.02
Step:  64400 Current evidence: -134.35
Step:  64500 Current evidence: -130.38
Step:  64600 Current evidence: -128.98
Step:  64700 Current evidence: -134.28
Step:  64800 Current evidence: -138.62
Step:  64900 Current evidence: -137.00
Step:  65000 Current evidence: -131.23
Step:  65100 Current evidence: -131.36
Step:  65200 Current evidence: -140.48
Step:  65300 Current evidence: -126.54
Step:  65400 Current evidence: -130.76
Step:  65500 Current evidence: -134.11
Step:  65600 Current evidence: -131.40
Step:  65700 Current evid

In [8]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

del gpr, pred, kernel
th.cuda.empty_cache()

tensor(3.9547, device='cuda:0', dtype=torch.float64)


## Titsias' variational EM GPR: m=250, 2.8788

In [ ]:
from aslearn.nonparametric.vigpr import VariationalEMSparseGPR
from aslearn.kernel.kernels import White, RBF

l = np.ones([21, 1]) * 1.
sigma = np.ones([1,]) * 1.
c = np.ones([1,]) * 0.1

white_kernel = White(c=c, dim_in=21, dim_out=1)
kernel = RBF(sigma=sigma, l=l, dim_in=21, dim_out=1)

gpr = VariationalEMSparseGPR(kernel=kernel, white_kernle=white_kernel)
ind = gpr.fit(X_train, Y_train, m=20, subsetNum=15)

Current mean elbo: -718055.41:  85%|██████████▏ | 17/20 [07:03<01:21, 27.27s/it]

In [7]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

del gpr, pred, kernel
th.cuda.empty_cache()

tensor(9.2787, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward0>)
